<h1>LECTURA O POBLACIÓN DE DATOS DE API A MONGODB</h1>
<p>Autores: CARLOS BREUER & EMILIO DELGADO (USO DE PYMONGO)</p>
<p>API utilizada: <a href="https://datos.madrid.es/portal/site/egob/menuitem.214413fe61bdd68a53318ba0a8a409a0/?vgnextoid=b07e0f7c5ff9e510VgnVCM1000008a4a900aRCRD&vgnextchannel=b07e0f7c5ff9e510VgnVCM1000008a4a900aRCRD&vgnextfmt=default">Portal de datos libres de Madrid</a></p>
<p>Cabe destacar que antes de ejecutar este script será necesario haber creado la colección CultureEvents sobre la base de datos 'Local' de MongoDB y haber creado un índice secundario y otro geoespacial de la siguiente manera:</p>
<p>db.CultureEvents.createIndex({"id":1},{ unique: true } );</p>
<p>db.CultureEvents.createIndex({'location':"2dsphere"});</p>

# LECTURA DE DATOS DE LA API

In [1]:
"1. Importación de librerías necesarias"
import requests 
import numpy as np 
import pandas as pd
from pymongo.errors import BulkWriteError

In [2]:
"2. Petición de tipo GET desde la capa REST"
r = requests.get('https://datos.madrid.es/egob/catalogo/206974-0-agenda-eventos-culturales-100.json')

In [3]:
"3. Obtención del contenido de la petición realizada en forma de diccionario (JSON) y se muestra su contenido"
my_dict = r.json()
my_dict

{'@context': {'c': 'http://www.w3.org/2002/12/cal#',
  'dcterms': 'http://purl.org/dc/terms/',
  'geo': 'http://www.w3.org/2003/01/geo/wgs84_pos#',
  'loc': 'http://purl.org/ctic/infraestructuras/localizacion#',
  'org': 'http://purl.org/ctic/infraestructuras/organizacion#',
  'vcard': 'http://www.w3.org/2006/vcard/ns#',
  'schema': 'https://schema.org/',
  'title': 'vcard:fn',
  'id': 'dcterms:identifier',
  'relation': 'dcterms:relation',
  'references': 'dcterms:references',
  'address': 'vcard:adr',
  'area': 'loc:barrio',
  'district': 'loc:distrito',
  'locality': 'vcard:locality',
  'postal-code': 'vcard:postal-code',
  'street-address': 'vcard:street-address',
  'location': 'vcard:geo',
  'latitude': 'geo:lat',
  'longitude': 'geo:long',
  'organization': 'vcard:org',
  'organization-desc': 'dcterms:description',
  'accesibility': 'org:accesibilidad',
  'services': 'org:servicios',
  'schedule': 'org:horario',
  'organization-name': 'vcard:organization-name',
  'description': '

In [4]:
"4. Transformación del diccionario en un dataframe de Pandas."
df = pd.DataFrame(my_dict['@graph'])

In [5]:
"5. Se realiza una copia del dataframe para procesar los datos sobre la copia en lugar de sobre el dataframe original"
df_copy = df.copy()

In [6]:
"6. Se muestra información sobre las columnas del dataframe"
df_copy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 970 entries, 0 to 969
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   @id             970 non-null    object
 1   @type           914 non-null    object
 2   id              970 non-null    object
 3   title           970 non-null    object
 4   description     970 non-null    object
 5   free            970 non-null    int64 
 6   price           970 non-null    object
 7   dtstart         970 non-null    object
 8   dtend           970 non-null    object
 9   time            970 non-null    object
 10  excluded-days   970 non-null    object
 11  uid             970 non-null    object
 12  link            970 non-null    object
 13  event-location  970 non-null    object
 14  references      970 non-null    object
 15  relation        892 non-null    object
 16  address         892 non-null    object
 17  location        892 non-null    object
 18  organizati

# FASE DE PRE-PROCESADO

In [7]:
"7. Los valores nulos de la columna AUDIENCE se sustituyen por la cadena 'No audience'"
df_copy['audience'].fillna('No audience',inplace=True)
"8. Los valores nulos de la columna @Type se sustituyen por una serie del tipo 'object'"
df_copy['@type'].fillna(pd.Series(dtype=object), inplace=True)
"9. Se eliminan todas las tuplas o filas cuyos valores en el campo 'location' sean nulos, es decir, no existe ubicación"
df_clean = df_copy[(df_copy['location'].isna()==False)]

In [8]:
"10. Se vuelve a mostrar información sobre las columnas del dataframe, esta vez ya pre-procesado"
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 892 entries, 0 to 969
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   @id             892 non-null    object
 1   @type           844 non-null    object
 2   id              892 non-null    object
 3   title           892 non-null    object
 4   description     892 non-null    object
 5   free            892 non-null    int64 
 6   price           892 non-null    object
 7   dtstart         892 non-null    object
 8   dtend           892 non-null    object
 9   time            892 non-null    object
 10  excluded-days   892 non-null    object
 11  uid             892 non-null    object
 12  link            892 non-null    object
 13  event-location  892 non-null    object
 14  references      892 non-null    object
 15  relation        892 non-null    object
 16  address         892 non-null    object
 17  location        892 non-null    object
 18  organizati

In [9]:
"11. Se muestran las primeras 20 filas del dataframe pre-procesado"
df_clean.head(20)

,@id,@type,id,title,description,free,price,dtstart,dtend,time,...,uid,link,event-location,references,relation,address,location,organization,audience,recurrence
0,https://datos.madrid.es/egob/catalogo/tipo/eve...,https://datos.madrid.es/egob/kos/actividades/T...,11673268,15 años con Miscelénea,Las amigas de Miscelánea celebran con nosotros...,1,,2022-05-27 19:00:00.0,2022-05-27 23:59:00.0,19:00,...,11673268,http://www.madrid.es/sites/v/index.jsp?vgnextc...,Centro Cultural Fernando de los Ríos (Latina),{'@id': ''},{'@id': 'https://datos.madrid.es/egob/catalogo...,{'district': {'@id': 'https://datos.madrid.es/...,"{'latitude': 40.39552931214334, 'longitude': -...",{'organization-name': 'Centro Cultural Fernand...,No audience,NaN
1,https://datos.madrid.es/egob/catalogo/tipo/eve...,https://datos.madrid.es/egob/kos/actividades/C...,11681414,"17 de mayo, Día Internacional contra la LGTBIF...",,1,,2022-05-17 18:00:00.0,2022-05-17 23:59:00.0,18:00,...,11681414,http://www.madrid.es/sites/v/index.jsp?vgnextc...,Centro Cultural Las Californias (Retiro),{'@id': 'http://www.madrid.es/sites/v/index.js...,{'@id': 'https://datos.madrid.es/egob/catalogo...,{'district': {'@id': 'https://datos.madrid.es/...,"{'latitude': 40.397020792322316, 'longitude': ...",{'organization-name': 'Centro Cultural Las Cal...,No audience,NaN
2,https://datos.madrid.es/egob/catalogo/tipo/eve...,https://datos.madrid.es/egob/kos/actividades/C...,11675246,"25 de Mayo, Día de África",,1,,2022-05-25 18:00:00.0,2022-05-25 23:59:00.0,18:00,...,11675246,http://www.madrid.es/sites/v/index.jsp?vgnextc...,Biblioteca Pública Municipal Iván de Vargas (C...,{'@id': ''},{'@id': 'https://datos.madrid.es/egob/catalogo...,{'district': {'@id': 'https://datos.madrid.es/...,"{'latitude': 40.41410290844665, 'longitude': -...",{'organization-name': 'Biblioteca Pública Muni...,Niños,NaN
3,https://datos.madrid.es/egob/catalogo/tipo/eve...,https://datos.madrid.es/egob/kos/actividades/P...,11666789,327 Cuadernos,,0,,2022-05-22 20:00:00.0,2022-05-22 23:59:00.0,20:00,...,11666789,http://www.madrid.es/sites/v/index.jsp?vgnextc...,Cineteca Madrid,{'@id': 'http://www.madrid.es/sites/v/index.js...,{'@id': 'https://datos.madrid.es/egob/catalogo...,{'district': {'@id': 'https://datos.madrid.es/...,"{'latitude': 40.391053041229135, 'longitude': ...","{'organization-name': 'Cineteca Madrid', 'acce...",No audience,NaN
4,https://datos.madrid.es/egob/catalogo/tipo/eve...,https://datos.madrid.es/egob/kos/actividades/F...,11671104,400 aniversario de la canonización de San Isidro,,1,,2022-05-06 00:00:00.0,2022-05-22 23:59:00.0,,...,11671104,http://www.madrid.es/sites/v/index.jsp?vgnextc...,Centro Cultural Fernando Lázaro Carreter (Cara...,{'@id': ''},{'@id': 'https://datos.madrid.es/egob/catalogo...,{'district': {'@id': 'https://datos.madrid.es/...,"{'latitude': 40.39164343238081, 'longitude': -...",{'organization-name': 'Centro Cultural Fernand...,No audience,"{'days': 'MO,TU,WE,TH,FR,SA,SU', 'frequency': ..."
6,https://datos.madrid.es/egob/catalogo/tipo/eve...,https://datos.madrid.es/egob/kos/actividades/C...,11660289,6 a 9 años. 'Aladdin' &amp; 'Peter Rabbit',,1,,2022-05-21 12:00:00.0,2022-05-21 23:59:00.0,12:00,...,11660289,http://www.madrid.es/sites/v/index.jsp?vgnextc...,Biblioteca Pública Municipal de San Fermín (Us...,{'@id': 'http://www.madrid.es/sites/v/index.js...,{'@id': 'https://datos.madrid.es/egob/catalogo...,{'district': {'@id': 'https://datos.madrid.es/...,"{'latitude': 40.3711623444796, 'longitude': -3...",{'organization-name': 'Biblioteca Pública Muni...,"Niños,Familias",NaN
7,https://datos.madrid.es/egob/catalogo/tipo/eve...,https://datos.madrid.es/egob/kos/actividades/C...,11660285,6 a 9 años. 'Aladdin' &amp; 'Peter Rabbit',,1,,2022-05-18 18:00:00.0,2022-05-18 23:59:00.0,18:00,...,11660285,http://www.madrid.es/sites/v/index.jsp?vgnextc...,Biblioteca Pública Municipal Eugenio Trías. Ca...,{'@id': 'http://www.madrid.es/sites/v/index.js...,{'@id': 'https://datos.madrid.es/egob/catalogo...,{'

# INSERCIÓN DE LOS DOCUMENTOS LEÍDOS DE LA API

In [10]:
"12. Definición de la función para realizar la conexión con la base de datos"
def get_db(CONNECTION_STRING):
    from pymongo import MongoClient
    import pymongo
    
    client = MongoClient(CONNECTION_STRING)
    return client    

In [11]:
"13. Establecimiento de la conexión con MongoDB y acceso a la base de datos 'local'"
#Del cliente se selecciona la base de datos que se desea, en este caso la local
CONNECTION_STRING = 'mongodb://localhost:27017/?readPreference=primary&appname=MongoDB%20Compass&directConnection=true&ssl=false'
db = get_db(CONNECTION_STRING)['local']
print(db)

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True, readpreference='primary', appname='MongoDB Compass', directconnection=True, tls=False), 'local')


In [12]:
"14. Acceso a la colección 'CultureEvents' de la base de datos 'local'. Se muestra información de la colección"
collection_name = db['CultureEvents']
print(collection_name)

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True, readpreference='primary', appname='MongoDB Compass', directconnection=True, tls=False), 'local'), 'CultureEvents')


In [13]:
"15. Se insertan las filas sin duplicados de aquellos nuevos eventos culturales que se hayan recuperado de la API y preprocesado previamente"
try:
    db.CultureEvents.insert_many(df_clean.to_dict('records'), ordered = False)
except BulkWriteError:
    pass